#sample brocker with


""" mosquitto_sub -h 127.0.0.1 -p 1883 -t test-topic -v """

test
"""mosquitto_pub -h 127.0.0.1 -t test-topic -m '{"ping":"pong"}'"""

In [1]:
#python3 -m pip install --user "paho-mqtt>=2.0"

In [ ]:
#!/usr/bin/env python3

import threading
import time
import cv2
from bbox_display import TrackerScene
from utils_deepstream import parse_ds_payload
from config import BrokerConfig as cfg
from utils_brocker_mqtt import MqttSubscriber

FPS = 60
WINDOW_NAME = "BBox Viewer"

scene = TrackerScene()

# Thread: רענון חלון כל הזמן – לא תלוי בהודעות
def draw_loop():
    delay = 1.0 / FPS
    while True:
        frame = scene.draw_scene()
        cv2.imshow(WINDOW_NAME, frame)
        key = cv2.waitKey(int(delay * 1000))
        if key == 27:  # ESC
            print("[EXIT] ESC pressed")
            break
    cv2.destroyAllWindows()
    exit(0)

# Callback MQTT: רק עדכון מופעים
def on_message(client, userdata, msg):
    try:
        objs = parse_ds_payload(msg.payload)
        ts_str = None
        if isinstance(objs, list) and objs:
            ts_str = objs[0].get("last_seen")
        print(f"[MQTT] {msg.topic}: {len(objs)} obj(s) ts={ts_str}")
        if objs:
            # מעבירים גם ts_str כדי לשמור להצגה
            scene.update_from_objs(objs, ts_str)
    except Exception as e:
        print(f"[ERROR] in on_message: {e}")

if __name__ == "__main__":
    # Thread ציור
    threading.Thread(target=draw_loop, daemon=True).start()

    # MQTT התחברות
    client = MqttSubscriber(cfg(), on_message=on_message).connect()
    try:
        client.start_forever()
    except KeyboardInterrupt:
        print("\n[MQTT] Stopped by user.")
    finally:
        client.stop()


[MQTT] Connecting to 127.0.0.1:1883 … (topics=['test-topic'])
[MQTT] on_connect: Connected
[MQTT] Subscribed to 'test-topic' (qos=0)
[DRAW] Frame with 0 object(s) at 1757445075.35
[DRAW] Frame with 0 object(s) at 1757445075.40
[DRAW] Frame with 0 object(s) at 1757445075.42
[DRAW] Frame with 0 object(s) at 1757445075.44
[DRAW] Frame with 0 object(s) at 1757445075.46
[DRAW] Frame with 0 object(s) at 1757445075.47
[DRAW] Frame with 0 object(s) at 1757445075.49
[DRAW] Frame with 0 object(s) at 1757445075.51
[DRAW] Frame with 0 object(s) at 1757445075.53
[DRAW] Frame with 0 object(s) at 1757445075.54
[DRAW] Frame with 0 object(s) at 1757445075.56
[DRAW] Frame with 0 object(s) at 1757445075.58
[DRAW] Frame with 0 object(s) at 1757445075.59
[DRAW] Frame with 0 object(s) at 1757445075.61
[DRAW] Frame with 0 object(s) at 1757445075.63
[DRAW] Frame with 0 object(s) at 1757445075.64
[DRAW] Frame with 0 object(s) at 1757445075.66
[DRAW] Frame with 0 object(s) at 1757445075.68
[DRAW] Frame with 0 o

: 

In [ ]:
#